In [2]:
import pandas as pd
import sqlite3

In [3]:
# conectamos con la base de datos my_database.db
connection = sqlite3.connect("database_cervezas.db")

# obtenemos un cursor que utilizamos para las queries
crsr = connection.cursor()


In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [5]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

cervezas
bares
empleados
reparto


In [6]:
query = '''
CREATE TABLE IF NOT EXISTS cervezas (
    CodC VARCHAR(2),
    Envase VARCHAR(32),
    Capacidad FLOAT(2),
    Stock INT(5),
    PRIMARY KEY (CodC)
)
'''

crsr.execute(query)

In [7]:
query = '''
CREATE TABLE IF NOT EXISTS bares (
    CodB VARCHAR(2),
    Nombre VARCHAR(32),
    Cif VARCHAR(32),
    Localidad VARCHAR(32),
    PRIMARY KEY (CodB)
)
'''

crsr.execute(query)

In [8]:
query = '''
CREATE TABLE IF NOT EXISTS empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(32),
    Sueldo INT(5),
    PRIMARY KEY (CodE)
)
'''

crsr.execute(query)

In [9]:
query = '''
DROP TABLE IF EXISTS reparto;
'''
crsr.execute(query)

In [10]:
query = '''
CREATE TABLE IF NOT EXISTS reparto (
    CodE VARCHAR(2) NOT NULL,
    CodB VARCHAR(2) NOT NULL,
    CodC VARCHAR(2) NOT NULL,
    Fecha DATE NOT NULL,
    Cantidad SMALLINT,
    PRIMARY KEY  (CodE,CodB,CodC,fecha),
    FOREIGN KEY (CodE) REFERENCES empleados(CodE),
    FOREIGN KEY (CodB) REFERENCES bares(CodB),
    FOREIGN KEY (CodC) REFERENCES cervezas(CodC)
);
'''
crsr.execute(query)

In [11]:
query = '''
INSERT OR REPLACE INTO empleados VALUES ('1','Carlos Lopez',120000),
('2', 'Rosa Perez', 110000),
('3', 'Luisa Garcia', 100000)
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [12]:
query = '''
INSERT OR REPLACE INTO bares VALUES ('001','Stop', '11111111X','Villa Botijo'),
('002', 'Las Vegas', '22222222Y', 'Villa Botijo'),
('003', 'Club Social', NULL, 'Las Ranas'),
('004','Otra Ronda','33333333Z','La Esponja')
'''
crsr.execute(query)

In [13]:
query = '''
INSERT OR IGNORE INTO cervezas VALUES ('01','Botella',0.2,3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04','Botella',1,288),
('05','Barril',60,30)
'''
crsr.execute(query)

In [14]:
query = '''
INSERT OR IGNORE INTO reparto VALUES 
('1', '001', '01', '2005-10-21', 240),
('1', '001', '02', '2005-10-21', 48),
('1', '002', '03', '2005-10-22', 60),
('1', '004', '05', '2005-10-22', 4),
('2', '002', '03', '2005-10-22', 48),
('2', '002', '05', '2005-10-23', 2),
('2', '004', '01', '2005-10-23', 480),
('2', '004', '02', '2005-10-24', 72),
('3', '003', '03', '2005-10-24', 48),
('3', '003', '04', '2005-10-25', 20)
'''

crsr.execute(query)

In [26]:
query = '''
SELECT * FROM reparto
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20


In [15]:
# 1 Obtener el nombre de los empleados que hayan repartido al bar Stop durante la semana 
# del 17 al 23 de octubre de 2005.

query = '''
SELECT DISTINCT empleados.Nombre
FROM reparto 
LEFT JOIN bares
ON reparto.CodB = bares.CodB
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
WHERE bares.Nombre = "Stop" AND reparto.Fecha BETWEEN '17/10/2005' AND '23/10/2005'
'''

sql_query(query)

,Nombre
0,Carlos Lopez


In [16]:
#2
query = '''
SELECT DISTINCT bares.Cif, bares.Nombre
FROM reparto 
LEFT JOIN bares
ON reparto.CodB = bares.CodB
LEFT JOIN cervezas
ON reparto.CodC = cervezas.CodC
WHERE cervezas.Envase = "Botella" AND cervezas.Capacidad < 1
ORDER BY bares.Localidad
'''

sql_query(query)

,Cif,Nombre
0,33333333Z,Otra Ronda
1,11111111X,Stop


In [17]:
#3 3. Obtener  los  repartos  (nombre  del  bar,  envase  y  
# capacidad  de  la  bebida, fecha y cantidad) realizados 
# por Carlos Lopez
query = '''
SELECT DISTINCT bares.Nombre, cervezas.Envase, cervezas.Capacidad, reparto.Fecha, reparto.Cantidad
FROM reparto 
LEFT JOIN bares
ON reparto.CodB = bares.CodB
LEFT JOIN cervezas
ON reparto.CodC = cervezas.CodC

'''

sql_query(query)

,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,2005-10-21,240
1,Stop,Botella,0.33,2005-10-21,48
2,Las Vegas,Lata,0.33,2005-10-22,60
3,Otra Ronda,Barril,60.00,2005-10-22,4
4,Las Vegas,Lata,0.33,2005-10-22,48
5,Las Vegas,Barril,60.00,2005-10-23,2
6,Otra Ronda,Botella,0.20,2005-10-23,480
7,Otra Ronda,Botella,0.33,2005-10-24,72
8,Club Social,Lata,0.33,2005-10-24,48
9,Club Social,Botella,1.00,2005-10-25,20


In [27]:
#4 Obtener  los  bares  a  los  que  se  les  ha  repartido  envases 
#  de  tipo  botella  y  capacidad  0.2  ó  0.33
query = '''
SELECT DISTINCT bares.Nombre, cervezas.Envase, cervezas.Capacidad
FROM reparto
LEFT JOIN bares
ON reparto.CodB = bares.CodB
LEFT JOIN cervezas
ON reparto.CodC = cervezas.CodC
WHERE cervezas.Envase = "Botella" AND cervezas.Capacidad IN (0.2, 0.33)
'''
sql_query(query)

,Nombre,Envase,Capacidad
0,Stop,Botella,0.20
1,Stop,Botella,0.33
2,Otra Ronda,Botella,0.20
3,Otra Ronda,Botella,0.33


In [28]:
#5 Nombre de los empleados que han repartido a los bares 
# "Stop" y "Las Vegas" cervezas con envase botella
query = '''
SELECT bares.Nombre, empleados.Nombre, cervezas.Envase
FROM reparto
LEFT JOIN bares
ON reparto.CodB = bares.CodB
LEFT JOIN cervezas
ON reparto.CodC = cervezas.CodC
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
WHERE bares.Nombre IN ("Stop", "Las Vegas") AND cervezas.Envase = "Botella"
'''
sql_query(query)

,Nombre,Nombre,Envase
0,Stop,Carlos Lopez,Botella
1,Stop,Carlos Lopez,Botella


In [20]:
#6
query = '''
SELECT empleados.CodE, empleados.Nombre, COUNT(*) 'Viajes'
FROM reparto
LEFT JOIN bares
ON reparto.CodB = bares.CodB
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
WHERE bares.localidad <> "Villa Botijo"
GROUP BY empleados.CodE

'''
sql_query(query)

,CodE,Nombre,Viajes
0,1,Carlos Lopez,1
1,2,Rosa Perez,2
2,3,Luisa Garcia,2


In [21]:
#7 Obtener el nombre y localidad del bar que más litros 
# de cerveza ha comprado.
query = '''
SELECT bares.Nombre, bares.Localidad, SUM(reparto.Cantidad * cervezas.Capacidad) AS Litros
FROM reparto
LEFT JOIN bares 
ON reparto.CodB = bares.CodB
LEFT JOIN cervezas 
ON reparto.CodC = cervezas.CodC
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 1
'''
sql_query(query)

,Nombre,Localidad,Litros
0,Otra Ronda,La Esponja,359.76


In [22]:
#8 Obtener  los  bares  que  han  adquirido  todos  los  tipos  
# de  cerveza  con  envase  de  botella   y capacidad menor 
# que 1 litro
query = '''
SELECT DISTINCT bares.Nombre 
FROM reparto
LEFT JOIN bares 
ON reparto.CodB = bares.CodB
LEFT JOIN cervezas 
ON reparto.CodC = cervezas.CodC
WHERE cervezas.Envase = "Botella" AND cervezas.Capacidad < 1
'''
sql_query(query)

,Nombre
0,Stop
1,Otra Ronda


In [23]:
#9 Subir un 5% el sueldo del empleado que más días haya trabajado
query = '''
SELECT COUNT( DISTINCT reparto.Fecha) AS 'Dias_trabajados', empleados.CodE, empleados.Nombre
FROM reparto
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
GROUP BY empleados.CodE
ORDER BY 1 DESC
'''
sql_query(query)

,Dias_trabajados,CodE,Nombre
0,3,2,Rosa Perez
1,2,3,Luisa Garcia
2,2,1,Carlos Lopez


In [24]:
query = '''
SELECT T.Sueldo *1.05
FROM ( SELECT COUNT( DISTINCT reparto.Fecha) AS 'Dias_trabajados', empleados.CodE, empleados.Nombre, empleados.Sueldo
FROM reparto
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
GROUP BY empleados.CodE
ORDER BY 1 DESC
LIMIT 1) t
'''
sql_query(query)

,T.Sueldo *1.05
0,115500.0


In [29]:
query = '''
SELECT t.CodE FROM
(
SELECT COUNT(DISTINCT reparto.Fecha) AS 'Dias_trabajados', empleados.CodE,  empleados.Sueldo
FROM reparto
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
GROUP BY empleados.CodE
ORDER BY 1 DESC
LIMIT 1) t     
'''
sql_query(query)

,CodE
0,2


In [30]:
query = '''

UPDATE empleados
SET sueldo = sueldo * 1.05
WHERE CodE IN (
SELECT t.CodE FROM
(
SELECT COUNT(DISTINCT reparto.Fecha) AS 'Dias_trabajados', empleados.CodE,  empleados.Sueldo
FROM reparto
LEFT JOIN empleados
ON reparto.CodE = empleados.CodE
GROUP BY empleados.CodE
ORDER BY 1 DESC
LIMIT 1) t     
)
'''
crsr.execute(query)

In [31]:
query = '''
SELECT *
FROM empleados
WHERE nombre = "Rosa Perez"
'''
sql_query(query)

,CodE,Nombre,Sueldo
0,2,Rosa Perez,115500


In [ ]:
connection.commit()
connection.close()